In [1]:
import sys
sys.path.append("../src/")

In [2]:
%load_ext autoreload
%autoreload 2
import data_utils as du

In [3]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


In [4]:
data = du.get_wiki_data(5)

INFO:absl:No config specified, defaulting to first: wiki40b/en
INFO:absl:Load dataset info from gs://tfds-data/datasets/wiki40b/en/1.3.0
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.splits from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.module_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset wiki40b (gs://tfds-data/datasets/wiki40b/en/1.3.0)
INFO:absl:Constructing tf.data.Dataset for split train[:5], from gs://tfds-data/datasets/wiki40b/en/1.3.0


In [ ]:
paragraphs = []
for d in data:
    text = d['text'].numpy().decode('utf-8')
    for para in text.split("_START_"):
        if not para.startswith("PARAGRAPH_"):
            continue
        proced_text = du.preproc_text(para)
        if len(proced_text) < 500:
            continue
        paragraphs.append(proced_text)
        # print(proced_text)
        # print(len(proced_text))
        
        #print("~"*20)
    #print("#"*20)
print(len(paragraphs))

In [ ]:
print(paragraphs[100])

# Format
* We need to keep both paragraph level and document level examples together.
* We can train consecutively any paragraph but also we can train any document with multiple paragraphs

## First try
* Split by paragraph
* Filter for paragraph length (above some length)
* We get roughly 1 paragraph over 1000 characters per document. That's not great.
* 789 / 500 docs with cutoff at 500 characters

# TODO
* Upgrade Spacy en_core_web_sm to latest
* fix data encoder error

In [ ]:
reprs = du.encode_data_srl_only(data[:10])

In [ ]:
import sys

In [ ]:
sys.getsizeof(data[:10])

In [ ]:
sys.getsizeof(reprs)

In [ ]:
len(reprs)

In [ ]:
reprs[0]

In [29]:
from collections import defaultdict

def build_repr(parse, word2veclist):
    unique_tags = set()
    sent_repr = []
    for verb in parse:
        tag_2_vec = defaultdict(list)
        assert len(verb['tags']) == len(word2veclist.keys())
        for i, tag in enumerate(verb['tags']):
            if tag == 'O' or tag[2] == "R" or tag[2] == "C":
                continue
            unique_tags.add(tag[2:])
            for vec_index in word2veclist[i]:
                tag_2_vec[tag.split("-")[1]].append(vec_index)
        for key in tag_2_vec.keys():
            assert key in du.TAGS, key +  str(list(unique_tags))
        verb_repr = []
        for tag in du.TAGS:
            if tag not in tag_2_vec:
                continue
            vecs = tag_2_vec[tag]
            verb_repr.append((tag, vecs))
        sent_repr.append(verb_repr)
    return sent_repr

In [166]:
def encode_data_srl_only(data):
    nlp = du.create_nlp()
    tokenizer = du.BertTokenizer.from_pretrained('bert-base-uncased')
    predictor = du.make_allennlp()
    reprs = []
    for ex_idx, d in enumerate(data):
        text = d['text'].numpy().decode('UTF-8').encode('ascii', 'ignore').decode('UTF-8')
        for para in text.split("_START_"):
            if not para.startswith("PARAGRAPH_"):
                continue
            example = du.preproc_text(para)
            if len(example) < 500:
                continue
            doc = nlp(example)
            for sent in doc.sents:
                span = sent.text
                lm_tokenized_input = tokenizer(span, return_tensors='pt')
                srl_parse = predictor.predict(sentence=span)
                token_strs = tokenizer.convert_ids_to_tokens(lm_tokenized_input['input_ids'].numpy()[0])
                try:
                    word2veclist = new_map_words_to_vectors(srl_parse['words'], token_strs)
                except du.NoStopException as e:
                    continue
                except Exception as e:
                    print("Index", ex_idx)
                    print(e)
                    print(token_strs)
                    print(srl_parse['words'])
                    raise e
                sent_repr = build_repr(srl_parse['verbs'], word2veclist)
                reprs.append((span, sent_repr))
    return reprs

In [31]:
reprs = encode_data_srl_only(data[:2])
print(reprs)

[('\nClayton was born in Mobile in south Alabama, but he was reared in Alexandria, Virginia.', [[('V', [2])], [('V', [3]), ('ARG1', [1]), ('ARGM', [4, 5, 6, 7, 8])], [('V', [12])], [('V', [13]), ('ARG1', [11]), ('ARGM', [14, 15, 16, 17])]]), ('His parents, Mr. and Mrs. William Gill Clayton, were Goldwater Republicans.', [[('V', [13]), ('ARG1', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]), ('ARG2', [14, 15, 16])]]), ('His father lobbied Congress on religious liberty issues.', [[('V', [3]), ('ARG0', [1, 2]), ('ARG1', [5, 6, 7, 8]), ('ARG2', [4])]]), ('Clayton graduated from high school and served in the United States Army Reserve.', [[('V', [2]), ('ARG0', [1]), ('ARG1', [3, 4, 5])], [('V', [7]), ('ARG0', [1]), ('ARG1', [8, 9, 10, 11, 12, 13])]]), ('He studied to be an aircraft electrician before he enrolled at Pensacola Christian College in Pensacola, Florida, from which he graduated in 2002.', [[('V', [2]), ('ARG0', [1]), ('ARG1', [3, 4, 5, 6, 7, 8]), ('ARGM', [9, 10, 11, 12, 13, 14, 15, 16

In [33]:
reprs[0]

('\nClayton was born in Mobile in south Alabama, but he was reared in Alexandria, Virginia.',
 [[('V', [2])],
  [('V', [3]), ('ARG1', [1]), ('ARGM', [4, 5, 6, 7, 8])],
  [('V', [12])],
  [('V', [13]), ('ARG1', [11]), ('ARGM', [14, 15, 16, 17])]])

In [36]:
tokenizer = du.BertTokenizer.from_pretrained('bert-base-uncased')
span = '\nClayton was born in Mobile in south Alabama, but he was reared in Alexandria, Virginia.'
lm_tokenized_input = tokenizer(span, return_tensors='pt')


In [37]:
lm_tokenized_input

{'input_ids': tensor([[  101, 11811,  2001,  2141,  1999,  4684,  1999,  2148,  6041,  1010,
          2021,  2002,  2001, 23295,  1999, 10297,  1010,  3448,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

# Fix parsing errors
* fix parser
* update spacy

In [43]:
data = du.get_wiki_data(500)

INFO:absl:No config specified, defaulting to first: wiki40b/en
INFO:absl:Load dataset info from gs://tfds-data/datasets/wiki40b/en/1.3.0
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.splits from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.module_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset wiki40b (gs://tfds-data/datasets/wiki40b/en/1.3.0)
INFO:absl:Constructing tf.data.Dataset for split train[:500], from gs://tfds-data/datasets/wiki40b/en/1.3.0


In [180]:
reprs = encode_data_srl_only(data[27:])

KeyboardInterrupt: 

In [182]:
reprs[33]

('Other demolished features include a barnyard where the Marchienne-au-Pont municipal swimming pool now stands.',
 [[('V', [2]), ('ARG1', [3])],
  [('V', [4]),
   ('ARG1', [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]),
   ('ARG2', [1, 2, 3])],
  [('V', [21]),
   ('ARG1', [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
   ('ARG2', [5, 6, 7]),
   ('ARGM', [20])]])

In [89]:
predictor = du.make_allennlp()

In [90]:
predictor.predict(sentence='\nClayton was born in Mobile in south Alabama, but he was reared in Alexandria, Virginia.'
)

{'verbs': [{'verb': 'was',
   'description': 'Clayton [V: was] born in Mobile in south Alabama , but he was reared in Alexandria , Virginia .',
   'tags': ['O',
    'B-V',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O']},
  {'verb': 'born',
   'description': '[ARG1: Clayton] was [V: born] [ARGM-LOC: in Mobile in south Alabama] , but he was reared in Alexandria , Virginia .',
   'tags': ['B-ARG1',
    'O',
    'B-V',
    'B-ARGM-LOC',
    'I-ARGM-LOC',
    'I-ARGM-LOC',
    'I-ARGM-LOC',
    'I-ARGM-LOC',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O']},
  {'verb': 'was',
   'description': 'Clayton was born in Mobile in south Alabama , but he [V: was] reared in Alexandria , Virginia .',
   'tags': ['O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'B-V',
    'O',
    'O',
    'O',
    'O',
    'O',
    

In [95]:
for ex_idx, d in enumerate(data[:1]):
    text = d['text'].numpy().decode('utf-8')
    for para in text.split("_START_"):
        if not para.startswith("PARAGRAPH_"):
            continue
        example = du.preproc_text(para)
        if len(example) < 500:
            continue
        doc = nlp(example)
        for sent in doc.sents:
            span = sent.text
            lm_tokenized_input = tokenizer(span, return_tensors='pt')
            srl_parse = predictor.predict(sentence=span)
            token_strs = tokenizer.convert_ids_to_tokens(lm_tokenized_input['input_ids'].numpy()[0])
            word2veclist = new_map_words_to_vectors(srl_parse['words'], token_strs)

['Kasturba', 'Road', 'is', 'a', 'street', 'in', 'Bangalore', ',', 'the', 'capital', 'of', 'Karnataka', ',', 'India', ',', 'which', 'is', 'connected', 'to', 'M', 'G', 'Road', 'to', 'the', 'north', 'and', 'J', 'C', 'Road', 'to', 'the', 'south', '.']
['[CLS]', 'ka', '##st', '##ur', '##ba', 'road', 'is', 'a', 'street', 'in', 'bangalore', ',', 'the', 'capital', 'of', 'karnataka', ',', 'india', ',', 'which', 'is', 'connected', 'to', 'm', 'g', 'road', 'to', 'the', 'north', 'and', 'j', 'c', 'road', 'to', 'the', 'south', '.', '[SEP]']
['Some', 'important', 'landmarks', 'situated', 'along', 'Kasturba', 'Road', 'are', 'Sree', 'Kanteerava', 'Stadium', ',', 'Kanteerava', 'Indoor', 'Stadium', ',', 'Cubbon', 'Park', ',', 'Government', 'Museum', ',', 'Venkatappa', 'Art', 'Gallery', ',', 'Visvesvaraya', 'Industrial', 'and', 'Technological', 'Museum', 'and', 'UB', 'City', '.']
['[CLS]', 'some', 'important', 'landmarks', 'situated', 'along', 'ka', '##st', '##ur', '##ba', 'road', 'are', 'sr', '##ee', 'kan

In [179]:
from collections import defaultdict

def count_chars(toks):
    count = 0
    for t  in toks:
        if t in ['[SEP]', '[CLS]']:
            continue
        count += len(t)
        if t.startswith("##"):
            count -= 2
    return count
            
def new_map_words_to_vectors(allen_srl_words, bert_tokenized_words):
    word2veclist = defaultdict(list)
    allen_srl_words = [w.lower() for w in allen_srl_words]
    all_srl_start_map = {}
    bert_tokenized_start_map = {}
#     print(allen_srl_words)
#     print(bert_tokenized_words)
    for i, word in enumerate(allen_srl_words):
        all_srl_start_map[count_chars(allen_srl_words[:i])] = (i, word)
    for i, word in enumerate(bert_tokenized_words):
        bert_tokenized_start_map[count_chars(bert_tokenized_words[:i])] = (i, word)
    # align all tokens with the same char_count and string
    for k, (srl_index, srl_word) in all_srl_start_map.items():
        if k in bert_tokenized_start_map and bert_tokenized_start_map[k][1] == srl_word:
            word2veclist[srl_index].append(bert_tokenized_start_map[k][0])
    
    for k, (srl_index, srl_word) in all_srl_start_map.items():
        if srl_index in word2veclist or k not in bert_tokenized_start_map:
            continue
        # matching start char_count but terms are different
        #print(srl_word, bert_tokenized_start_map[k][1])
        word2veclist[srl_index].append(bert_tokenized_start_map[k][0])
    completed_tokenized_indexes = set([v for vals in word2veclist.values() for v in vals])
    inverse_word2veclist = {}
    for k, v in word2veclist.items():
        for vv in v:
            inverse_word2veclist[vv] = k
    for bert_tok_index, bert_tok_word in enumerate(bert_tokenized_words):
        if bert_tok_index in completed_tokenized_indexes or bert_tok_word in ['[SEP]', '[CLS]']:
            continue
        # print("#"*20)
        # print(bert_tok_index, bert_tok_word)
        # if token before this token is completed and word index after is in word2veclist
        # just add this to the prev tok
        if (bert_tok_index - 1) in completed_tokenized_indexes:
#             print("Prev tok index is complete")
#             print("Prev Tok", bert_tok_index - 1, bert_tokenized_words[bert_tok_index - 1])
            srl_word_of_prev_tok = inverse_word2veclist[bert_tok_index - 1]
            # print("SRL word of prev tok:", allen_srl_words[srl_word_of_prev_tok])
            if (srl_word_of_prev_tok + 1) in word2veclist or srl_word_of_prev_tok + 1 ==  len(allen_srl_words):
                word2veclist[srl_word_of_prev_tok].append(bert_tok_index)
                inverse_word2veclist[bert_tok_index] = srl_word_of_prev_tok
                completed_tokenized_indexes.add(bert_tok_index)
                # print("Adding ", bert_tokenized_words[bert_tok_index], "to", allen_srl_words[srl_word_of_prev_tok])
    # At this point, the idea is that if there are still any srl_words not in the word2veclist
    # if their neighbors are in, then you can just map all the tokens not used between those two
    for srl_index, srl_word in enumerate(allen_srl_words):
        if srl_index not in word2veclist:
            if srl_index -1 in word2veclist and srl_index +1 in word2veclist:
                start = max(word2veclist[srl_index -1])
                end = min(word2veclist[srl_index +1])
                tok_indexes = list(range(start+1, end))
                if tok_indexes:
                    word2veclist[srl_index].extend(tok_indexes)
                else:
                    word2veclist[srl_index] = []
    
    try:
        if not len(word2veclist.keys()) == len(allen_srl_words):
            raise Exception
        if not len([v for vals in word2veclist.values() for v in vals]) == len(bert_tokenized_words) - 2:
            raise Exception("Er mergahd")
    except Exception as e:
        print(word2veclist)
        print(all_srl_start_map)
        print(bert_tokenized_start_map)
        raise e
    return  word2veclist
    

word2veclist = new_map_words_to_vectors(srl_parse['words'], token_strs)
print(word2veclist)
assert len(word2veclist.keys()) == len(srl_parse['words'])
assert len([v for vals in word2veclist.values() for v in vals]) == len(token_strs) - 2

defaultdict(<class 'list'>, {0: [1], 1: [2], 2: [3], 3: [4], 4: [5], 5: [6], 6: [7], 7: [8], 8: [9], 9: [10], 10: [11], 11: [12], 14: [18], 16: [23], 17: [24], 12: [13, 14], 13: [15, 16, 17], 15: [19, 20, 21, 22]})


In [167]:
data[6]['text'].numpy().decode('UTF-8').encode('ascii', 'ignore').decode('UTF-8').split("_START_")# .decode("utf-8")

['\n',
 'ARTICLE_\nFluor-liddicoatite\n',
 'SECTION_\nCrystal habit\n',
 'PARAGRAPH_\nCrystals are stout prismatic, with a curved convex trigonal outline, generally elongated and striated parallel to the c axis.  Crystals are hemimorphic, meaning that the two ends of the crystal have different forms.  Fluor-liddicoatite usually has a pedion (a single crystal face) opposite one or two pyramids.\n',
 'SECTION_\nPhysical properties\n',
 'PARAGRAPH_\nThe color is usually smoky brown, but also pink, red, green, blue, or rarely white.  Color zoning is abundant at the type locality, parallel to pyramid faces. This is due to changes in the solution during crystal growth. As the concentration of trace elements that serve as coloring agents changes, there will be areas of less or more color in different parts of the crystal. When the crystal is sliced perpendicular to the c axis, triangular zoning may be seen, together with a trigonal star that radiates from the centre of the crystal, with the t

In [155]:
type(data[6]['text'].numpy())

bytes